# Decision Trees

Decision tree is a type of **supervised learning** algorithm that works both for **regression** and **classification** problems. It works for both **categorical and continuous input and output variables**. In this technique, we **split** the population or sample **into two or more homogeneous sets** (or sub-populations) based on most significant splitter / differentiator in input variables. Decision Tree's are really popular because they are very easy to **interpret** and forms the basis for more complex bagging techniques such as **Random Forest**. 

**Why binary split are favoured?** <br>
Generally, we use binary tree to split the tree instead of n-split tree as binary tree's much more easy to implement and are very efficient. Also, any n-split can be easily achieved even with multiple binary splits.

## Key Terminologies
Let’s look at the basic terminology used with Decision trees:

**Root Node:** It represents entire population or sample and this further gets divided into two or more homogeneous sets.<br>
**Splitting:** It is a process of dividing a node into two or more sub-nodes. <br>
**Decision Node:** When a sub-node splits into further sub-nodes, then it is called decision node.<br>
**Leaf/ Terminal Node:** Nodes do not split is called Leaf or Terminal node.<br>
**Pruning:** When we remove sub-nodes of a decision node, this process is called pruning. You can say opposite process of splitting.<br>
**Branch / Sub-Tree:** A sub section of entire tree is called branch or sub-tree.<br>
**Parent and Child Node:** A node, which is divided into sub-nodes is called parent node of sub-nodes where as sub-nodes are the child of parent node.

![title](https://www.analyticsvidhya.com/wp-content/uploads/2015/01/Decision_Tree_2.png)

## Splitting Criterion
The decision of making strategic splits **heavily affects a tree’s accuracy**. The decision **criteria is different for classification and regression trees**.The creation of sub-nodes increases the **homogeneity** of resultant sub-nodes. In other words, we can say that **purity of the node increases** with respect to the target variable. Decision tree splits the nodes on **all available variables** and then selects the split which results in **most homogeneous sub-nodes**. Refer to following [blog](https://www.analyticsvidhya.com/blog/2016/04/complete-tutorial-tree-based-modeling-scratch-in-python/) for detailed explanation.

The algorithm selection is also based on **type of target variables**. Let’s look at the four most commonly used algorithms in decision tree:

**Gini Index**: Gini index says, if we select two items from a population at random then they must be of same class and probability for this is 1 if population is pure.
1. It works with categorical target variable “Success” or “Failure” and  performs only Binary splits

Steps to Calculate Gini for a split
1. Calculate Gini for sub-nodes, using formula sum of square of probability for success and failure (p^2+q^2).
2. Calculate Gini for split using weighted Gini score of each node of that split.<br>

![title](https://cdn-images-1.medium.com/max/800/1*2bI1Uxv3bBzR9Nkg_PLiaw.png)
![title](https://cdn-images-1.medium.com/max/800/1*kvIc3ZwDyYab3GOOKY5lHw.png)

**Node split happens for feature with higher Gini Score**

**Chi-Square**:It is an algorithm to find out the statistical significance between the differences between sub-nodes and parent node. We measure it by sum of squares of standardized differences between observed and expected frequencies of target variable.
1. It works with categorical target variable “Success” or “Failure” and can perform two or more splits.
2. Higher the value of Chi-Square higher the statistical significance of differences between sub-node and Parent node.
3. Chi-square = ((Actual – Expected)^2 / Expected)^1/2

Steps to Calculate Chi-square for a split:

1. Calculate Chi-square for individual node by calculating the deviation for Success and Failure both
2. Calculated Chi-square of Split using Sum of all Chi-square of success and Failure of each node of the split

**Information Gain:** Information gain is derived in terms of entropy. Entropy is the degree of disorganization in a system. If the sample is completely homogeneous, then the entropy is zero and if the sample is an equally divided (50% – 50%), it has entropy of one. Entropy is given by the formula:
$$ E = -p\log_2(p) - q\log_2(q)$$

Here p and q is probability of success and failure respectively in that node. Entropy is also used with categorical target variable. It chooses the split which has lowest entropy compared to parent node and other splits. The lesser the entropy, the better it is.

Steps to calculate entropy for a split:

1. Calculate entropy of parent node
2. Calculate entropy of each individual node of split and calculate weighted average of all sub-nodes available in split.
3. The difference between parent and sub-node is called Information gain.

**Reduction in Variance** Reduction in variance is an algorithm used for continuous target variables (regression problems). This algorithm uses the standard formula of variance to choose the best split. The split with lower variance is selected as the criteria to split the population:

$$ var = \sum\frac{(X-\bar(X))^2}{n-1}$$

Steps to calculate Variance:

1. Calculate variance for each node.
2. Calculate variance for each split as weighted average of each node variance.

## Stopping Criterion

Overfitting is one of the key challenges faced while modeling decision trees. Preventing overfitting is pivotal while modeling a decision tree and it can be done in 2 ways:

### 1. Setting constraints on tree size: 
This can be done by using various parameters which are used to define a tree. 

**Minimum samples for a node split**: Defines the minimum number of samples (or observations) which are required in a node to be considered for splitting.<br>
**Minimum samples for a terminal node (leaf)**:Defines the minimum samples (or observations) required in a terminal node or leaf.Generally lower values should be chosen for imbalanced class problems because the regions in which the minority class will be in majority will be very small. <br>
**Maximum depth of a tree**: Used to control over-fitting as higher depth will allow model to learn relations very specific to a particular sample.<br>
**Maximum number of terminal nodes**:The maximum number of terminal nodes or leaves in a tree. Can be defined in place of max_depth. Since binary trees are created, a depth of ‘n’ would produce a maximum of 2^n leaves.<br>
**Maximum features to consider for split:** The number of features to consider while searching for a best split. These will be randomly selected. As a thumb-rule, square root of the total number of features works great but we should check upto 30-40% of the total number of features.

### 2. Tree pruning:
We grow the tree and then remove splits which does not given good gains. Currently not present in sklearn. 

## Implementation of decision Tree
Key Components: 
1. Initialize a tree
2. Implement splitting criterion
3. Find best split for a feature
4. Find best split for across multiple features
4. Create branches/lower nodes
5. Prediction function

### Implementing Decision tree classifier with Gini Index
We will start by implementing a decision tree classifier with Gini Index. Splitting criterion can be changed easily by just changing splitting criterion function.

### 1. Initialize a tree
We **initialize** a tree with **max depth**, **current depth**, **left side node and right side node**. We **can add more** attributes if we want. For given implementation, we will go ahead with this. We **will slightly change it during Regression model** to show some variations:

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import mode

class DecisionTree:

    def __init__(self, max_depth = 5, depth = 1):
        self.max_depth = max_depth
        self.depth = depth
        self.left = None
        self.right = None

### 2. Implement splitting criterion(Gini Index)

In [2]:
def __calculate_impurity_score(self, data):
    if data is None or data.empty: return 0
    gini_impurity = 1.
    classes = np.unique(data)
    n_vals = len(data)
    for cl in classes:
        gini_impurity -= np.square(np.sum(data==cl)/n_vals)  # subtracting sum of square probability from 1
    return gini_impurity

Above is the impurity calculation. To complete the splitting, we will also need the information gain information, i.e weighted difference in Gini score between parent and child nodes. Note **__** is used before function name as they are meant to be **internal functions(in a sense private functions)** which we do not want to call from outside. Since, **python does not have** a concept of **private vs public functions**, we use following convention for ease of understanding. <br>

Here is the implementation for **Information Gain**. We **subtract** the **weighted child impurity score from Parent node**. `self.impurity_score` already has parent gini score stored(see in full code) 

In [3]:
def __calculate_information_gain(self, left_count, left_impurity, right_count, right_impurity):
    return self.impurity_score - ((left_count/len(self.data)) * left_impurity + \
                                  (right_count/len(self.data)) * right_impurity)

### 3. Find best split for a feature column

For tree splitting on a given column, we do following things:
1. Find the unique values in the column as we split on the column values.
2. If there is only 1 unique value, no need to split as it is already pure node.
2. Run a loop on all unique values as split point
3. Values less than equal to split goes to left and larger values goes to right. This way we create a binary split.
4. Calculate the impurity score and information gain based on above split.
5. Store the split which has maximum information gain.

**Note that information gain is always >= 0 **

In [4]:
def __find_best_split_for_column(self, col):
    x = self.data[col]
    unique_values = x.unique()
    if len(unique_values) == 1: return None, None
    information_gain = None
    split = None
    for val in unique_values:
        left = x <= val
        right = x > val
        left_data = self.data[left]
        right_data = self.data[right]
        left_impurity = self.__calculate_impurity_score(left_data[self.target])
        right_impurity = self.__calculate_impurity_score(right_data[self.target])
        score = self.__calculate_information_gain(left_count = len(left_data),
                                                  left_impurity = left_impurity,
                                                  right_count = len(right_data),
                                                  right_impurity = right_impurity)
        if information_gain is None or score > information_gain: 
            information_gain = score 
            split = val
    return information_gain, split

### 4. Find best split for across multiple features:

This is just a wrapper function on top of `__find_best_split_for_column` to find the best column and split value across different columns/features. Note that all features are available at every node although we split on specific column/feature.

In [5]:
def __find_best_split(self):
    best_split = {}
    for col in self.columns:
        information_gain, split = self.__find_best_split_for_column(col)
        if split is None: continue
        if not best_split or best_split["information_gain"] < information_gain:
            best_split = {"split": split, "col": col, "information_gain": information_gain}

    return best_split.get("split"), best_split.get("col"), best_split.get("information_gain")

### 5. Create branches/lower nodes

Once we have found the best column/feature and split value to split on, we split the node into two child nodes or we grow the tree downwards. Here are the steps:
1. Initialize two new nodes of type `DecisionTree` called as `left` and `right`.
2. Using split column/feature and split value, divide the data as `left_rows` and `right_rows`.
3. Call fit function to create and initialize tree branches.

We will discuss `self.fit` next.

In [6]:
def __create_branches(self):
    self.left = DecisionTree(max_depth = self.max_depth, 
                             depth = self.depth + 1)
    self.right = DecisionTree(max_depth = self.max_depth, 
                             depth = self.depth + 1)
    left_rows = self.data[self.data[self.split_feature] <= self.criteria] 
    right_rows = self.data[self.data[self.split_feature] > self.criteria] 
    self.left.fit(data = left_rows, target = self.target)
    self.right.fit(data = right_rows, target = self.target)

### Fit function
Once we have created the `DecisionTree` **constructor**, we need to call the **fit** function with the **data and target column** to start fitting a `DecisionTree` on the **training data**. Below is the definition of the fit function. It does following things:
1. It takes the `data` and `name of the target column(str)` as input.
2. It initializes `columns` with the name of the feature columns.
3. If `max depth` of the tree is not reached, it validates the data to check it is in correct format.
4. It calculates impurity score for the parent node and finds best split and performs splitting. 

In [7]:
def fit(self, data, target):
    self.data = data
    self.target = target
    self.columns = self.data.columns.tolist()
    self.columns.remove(target)
    if self.depth <= self.max_depth:
        self.__validate_data()
        self.impurity_score = self.__calculate_impurity_score(self.data[self.target])
        self.criteria, self.split_feature, self.information_gain = self.__find_best_split()
        if self.criteria is not None and self.information_gain > 0: self.__create_branches()
    else: 
        print("Stopping splitting as Max depth reached")

### Data validation function
**DecisionTree** needs all **columns values to numeric**. It check this condition and raises error if required

In [8]:
def __validate_data(self):
    non_numeric_columns = self.data[self.columns].select_dtypes(include=['category', 'object', 'bool']).columns.tolist()
    if(len(set(self.columns).intersection(set(non_numeric_columns))) != 0):
        raise RuntimeError("Not all columns are numeric")


### 6. Prediction function

In a decision tree, the **predictions are made at the leaves**, i.e nodes which has no childs. So, in order to make prediction for a test point, we need to traverse the tree based on split logics and reach the corresponding leaf. Once we reach leaf, we can make prediction in a variety of ways:
1. Predict based on majority voting of target observed during training
2. Take weighted vote
3. So other custom function to find the prediction. 

For now, we will go ahead with majority voting. In order to implement predict function, we first need following things:

**a) Check if a node is leaf or not**:<br>

We do this by defining a **class property** as shown below. It **checks** if the node has any **left branch** or not. It is also **equivalent** to **checking right or both** as we always do **binary splits.**

In [9]:
@property
def is_leaf_node(self): return self.left is None

**Note `property` can be thought as a way of using a function as an attribute which can help in getting(getter) or setting(setter) and value stored in the class object** Follow this [link](https://www.programiz.com/python-programming/property) to better understanding. 

**b) Majority vote as prediction**:<br?
We implement this as well using class property as shown below:

In [10]:
@property
def prediction(self): 
    return mode(self.data[self.target])[0][0]

**c) Walk through the tree till reach node and make prediction**
We define a function, which **recursively transverse** the tree and make prediction once leaf is reached as shown below. At every point we **check the splitting criterion and go to left or right** based on where our test point lies in feature space.

In [11]:
def __flow_data_thru_tree(self, row):
    if self.is_leaf_node: return self.prediction
    tree = self.left if row[self.split_feature] <= self.criteria else self.right
    return tree.__flow_data_thru_tree(row)

**d) Wrapper Function to predict for all test points**
Following function, makes prediction for all test points given for prediction.

In [12]:
def predict(self, data):
    return np.array([self.__flow_data_thru_tree(row) for _, row in data.iterrows()])

## Complete Implementation for DecisonTree Classifier

In [13]:
import numpy as np
import pandas as pd

class DecisionTree:

    def __init__(self, max_depth = 5, depth = 1):
        self.max_depth = max_depth
        self.depth = depth
        self.left = None
        self.right = None
    
    def fit(self, data, target):
        self.data = data
        self.target = target
        self.columns = self.data.columns.tolist()
        self.columns.remove(target)
        if self.depth <= self.max_depth:
            self.__validate_data()
            self.impurity_score = self.__calculate_impurity_score(self.data[self.target])
            self.criteria, self.split_feature, self.information_gain = self.__find_best_split()
            if self.criteria is not None and self.information_gain > 0: self.__create_branches()
        else: 
            print("Stopping splitting as Max depth reached")
    
    def __create_branches(self):
        self.left = DecisionTree(max_depth = self.max_depth, 
                                 depth = self.depth + 1)
        self.right = DecisionTree(max_depth = self.max_depth, 
                                 depth = self.depth + 1)
        left_rows = self.data[self.data[self.split_feature] <= self.criteria] 
        right_rows = self.data[self.data[self.split_feature] > self.criteria] 
        self.left.fit(data = left_rows, target = self.target)
        self.right.fit(data = right_rows, target = self.target)
    
    def __calculate_impurity_score(self, data):
        if data is None or data.empty: return 0
        gini_impurity = 1.
        classes = np.unique(data)
        n_vals = len(data)
        for cl in classes:
            gini_impurity -= np.square(np.sum(data==cl)/n_vals)  # substarcting sum of square probability from 1
        return gini_impurity
    
    def __find_best_split(self):
        best_split = {}
        for col in self.columns:
            information_gain, split = self.__find_best_split_for_column(col)
            if split is None: continue
            if not best_split or best_split["information_gain"] < information_gain:
                best_split = {"split": split, "col": col, "information_gain": information_gain}

        return best_split.get("split"), best_split.get("col"), best_split.get("information_gain")

    def __find_best_split_for_column(self, col):
        x = self.data[col]
        unique_values = x.unique()
        if len(unique_values) == 1: return None, None
        information_gain = None
        split = None
        for val in unique_values:
            left = x <= val
            right = x > val
            left_data = self.data[left]
            right_data = self.data[right]
            left_impurity = self.__calculate_impurity_score(left_data[self.target])
            right_impurity = self.__calculate_impurity_score(right_data[self.target])
            score = self.__calculate_information_gain(left_count = len(left_data),
                                                      left_impurity = left_impurity,
                                                      right_count = len(right_data),
                                                      right_impurity = right_impurity)
            if information_gain is None or score > information_gain: 
                information_gain = score 
                split = val
        return information_gain, split
    
    def __calculate_information_gain(self, left_count, left_impurity, right_count, right_impurity):
        return self.impurity_score - ((left_count/len(self.data)) * left_impurity + \
                                      (right_count/len(self.data)) * right_impurity)

    def predict(self, data):
        return np.array([self.__flow_data_thru_tree(row) for _, row in data.iterrows()])

    def __validate_data(self):
        non_numeric_columns = self.data[self.columns].select_dtypes(include=['category', 'object', 'bool']).columns.tolist()
        if(len(set(self.columns).intersection(set(non_numeric_columns))) != 0):
            raise RuntimeError("Not all columns are numeric")

    def __flow_data_thru_tree(self, row):
        if self.is_leaf_node: return self.probability
        tree = self.left if row[self.split_feature] <= self.criteria else self.right
        return tree.__flow_data_thru_tree(row)
        
    @property
    def is_leaf_node(self): return self.left is None

    @property
    def probability(self): 
        return mode(self.data[self.target])[0][0]


## Check implementation on iris data

In [14]:
from sklearn.datasets import load_iris

In [15]:
iris = load_iris()

In [16]:
iris_df = pd.DataFrame(iris.data, columns=iris.feature_names)
iris_df['target'] = iris.target

In [17]:
clf = DecisionTree(max_depth= 3)
clf.fit(iris_df, 'target')

Stopping splitting as Max depth reached
Stopping splitting as Max depth reached
Stopping splitting as Max depth reached
Stopping splitting as Max depth reached


In [18]:
print("Accuracy %.3f" %(np.sum(clf.predict(iris_df)==iris.target)*1.0/len(iris.target)))

Accuracy 0.973


### Comparing to Sklearn 

In [19]:
from sklearn.tree import DecisionTreeClassifier
skclf = DecisionTreeClassifier(max_depth=3)

In [20]:
skclf.fit(X=iris.data, y=iris.target)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=3,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [21]:
print("Accuracy %.3f" %(np.sum(skclf.predict(iris.data)==iris.target)*1.0/len(iris.target)))

Accuracy 0.973


**We can see that we get the same accuracy showing our implementation is correct.**

## Pros and Cons

**Pros**
1. Easy to Understand
2. Useful in Data exploration
3. Less data cleaning required
4. Data type is not a constraint
5. Non Parametric Method

**Cons**
1. Over fitting
2. Not great for continuous variables

Check this [blog](https://www.analyticsvidhya.com/blog/2016/04/complete-tutorial-tree-based-modeling-scratch-in-python/) for detailed explanation.

## Variations to try
1. **Regression:** We will implement in a different notebook
2. Try **different impurity** calculation like entropy
3. **Do beam search:** We have implemented a greedy algo which looks only at immediate split to find best split. We will implement beam search which looks at next `n` splits to find best split in a different notebook.